In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from tqdm import tqdm
import os
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from joblib import dump, load
import ltn
import csv
import math
import wandb


dataset_path = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU'

PGB_path = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/PGB/PGB'
RGB_path = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/RGB/RGB'

# Specify the CSV file path
csv_file = '/home/ubuntu/dds_paper/DDS_Paper/data/data_robust.csv'
preprocessor_file = 'preprocessor.joblib'

train_path = '/home/ubuntu/dds_paper/DDS_Paper/data/train.csv'
val_path = '/home/ubuntu/dds_paper/DDS_Paper/data/val.csv'

np.random.seed(45)

# Set the chunk size for reading the CSV
chunk_size = 100000  # Adjust the chunk size according to your memory limitations

2024-03-13 13:18:48.469604: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 13:18:48.483187: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 13:18:48.608732: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 13:18:50.223262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
import csv
import pandas as pd
from tqdm import tqdm

def extract_fault(file_name):
    fault_mapping = {
        '0Health': 'HEA', '1Chipped': 'CTF', '2Miss': 'MTF', 
        '3Root': 'RCF', '4Surface': 'SWF', '5Ball': 'BWF', 
        '6Combination': 'CWF', '7Inner': 'IRF', '8Outer': 'ORF'
    }
    for key, value in fault_mapping.items():
        if key in file_name:
            return value
    return None

def make_csv_writer(csv_file):
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8', 'Fault'])
    return csv_writer

def generate_csv(output_directory, root_path, speed, experiment, files):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    train_filename_suffix = f"{speed}_{experiment}_train" if experiment else f"{speed}_train"
    test_filename_suffix = f"{speed}_{experiment}_test" if experiment else f"{speed}_test"
    
    train_output_file_path = os.path.join(output_directory, f"PGB_{train_filename_suffix}.csv")
    test_output_file_path = os.path.join(output_directory, f"PGB_{test_filename_suffix}.csv")
    
    with open(train_output_file_path, 'w', newline='', encoding='utf-8') as train_csvfile, \
        open(test_output_file_path, 'w', newline='', encoding='utf-8') as test_csvfile:
        train_csv_writer = make_csv_writer(train_csvfile)
        test_csv_writer = make_csv_writer(test_csvfile)
        
        for file in tqdm(files, desc=f"Processing {speed} {experiment}", unit="file"):
            fault_type = extract_fault(file)
            # Only append 'speed' directory for non-variable speed cases
            if experiment:
                file_path = os.path.join(root_path, file)  # Already includes 'Variable_speed/Experiment#'
            else:
                file_path = os.path.join(root_path, file)  # 'root_path' already includes 'speed' directory
            
            data = pd.read_csv(file_path, sep='\t', header=None, encoding='ISO-8859-1', skiprows=1, nrows=100000)
            train_samples, test_samples = data.iloc[:80000, :], data.iloc[80000:100000, :]
            
            for index, row in train_samples.iterrows():
                train_csv_writer.writerow(row[:8].tolist() + [fault_type])
            
            for index, row in test_samples.iterrows():
                test_csv_writer.writerow(row[:8].tolist() + [fault_type])

def process_pgb_data(data_root_folder, output_directory):
    for root, dirs, files in os.walk(data_root_folder):
        parts = root.split(os.sep)
        if 'Variable_speed' in parts:
            speed = "Variable_speed"
            experiment_dir = parts[-1]  # Get the last part as the experiment name
            exp_files = [f for f in os.listdir(root) if f.endswith('.txt')]
            # Pass the 'root' directly without modifying it for variable speed
            generate_csv(output_directory, root, speed, experiment_dir, exp_files)
        elif 'PGB' in parts and files:
            speed = parts[-1]  # Last part of 'root' is the speed directory
            # For non-variable speed, pass the 'root' directly
            generate_csv(output_directory, root, speed, '', files)

data_root_folder = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/PGB/PGB'
output_directory = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/data/csvs'
process_pgb_data(data_root_folder, output_directory)



Processing Variable_speed Variable_speed: 0file [00:00, ?file/s]
Processing 30_1 : 100%|██████████| 9/9 [00:58<00:00,  6.53s/file]


In [3]:
import os
import pandas as pd
from collections import Counter

def overview_csv_files(directory):
    data = []
    all_faults = set()

    for file in os.listdir(directory):
        if file.endswith(".csv"):
            file_path = os.path.join(directory, file)
            df = pd.read_csv(file_path)

            # Check if the CSV is empty (aside from the header)
            if df.shape[0] == 0:
                # Delete the empty CSV file
                os.remove(file_path)
                print(f"Deleted empty file: {file_path}")
                continue  # Skip further processing for this file

            num_samples = len(df)
            fault_distribution = Counter(df['Fault'])
            all_faults.update(fault_distribution.keys())
            data.append({'File Name': file, 'Number of Samples': num_samples, **fault_distribution})

    if not data:  # If no data has been gathered, exit the function
        print("No data found.")
        return

    overview_df = pd.DataFrame(data)
    for fault in all_faults:
        if fault not in overview_df.columns:
            overview_df[fault] = 0

    cols = ['File Name', 'Number of Samples'] + sorted(all_faults)
    overview_df = overview_df[cols]
    overview_df.fillna(0, inplace=True)
    overview_df.loc[:, 'Number of Samples':] = overview_df.loc[:, 'Number of Samples':].astype(int)

    overview_df = overview_df.sort_values(by='File Name')
    print(overview_df.to_string(index=False))

# Example usage
directory = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/data/csvs'
overview_csv_files(directory)


Deleted empty file: /home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/data/csvs/PGB_Variable_speed_Variable_speed_train.csv
Deleted empty file: /home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/data/csvs/PGB_Variable_speed_Variable_speed_test.csv
                                File Name  Number of Samples   BWF   CTF   CWF   HEA   IRF   MTF   ORF   RCF   SWF
                        PGB_20_0_test.csv             180000 20000 20000 20000 20000 20000 20000 20000 20000 20000
                       PGB_20_0_train.csv             720000 80000 80000 80000 80000 80000 80000 80000 80000 80000
                        PGB_30_0_test.csv             180000 20000 20000 20000 20000 20000 20000 20000 20000 20000
                       PGB_30_0_train.csv             720000 80000 80000 80000 80000 80000 80000 80000 80000 80000
                        PGB_30_1_test.csv             180000 20000 20000 20000 20000 20000 20000 20000 20000 20000
                       PGB_30_1_train.csv             720000 

In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import joblib  # For saving the scaler model

def load_and_scale_data(csv_path, scaler=None, save_scaler_path=None):
    """
    Loads data from a CSV file, scales the features (excluding the 'Fault' column), 
    and returns the scaled DataFrame. Optionally saves the scaler model.
    """
    # Load the data
    data = pd.read_csv(csv_path)
    
    # Separate features and target
    features = data.columns[:-1]  # Assuming the last column is the target
    X = data[features]
    y = data['Fault']

    # Apply scaling
    if scaler is None:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        if save_scaler_path:
            joblib.dump(scaler, save_scaler_path)
    else:
        X_scaled = scaler.transform(X)
    
    # Combine scaled features with target
    scaled_df = pd.DataFrame(X_scaled, columns=features)
    scaled_df['Fault'] = y
    
    return scaled_df

# Define your dataset directory
dataset_dir = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/data/csvs'

# Iterate over your dataset files
for root, dirs, files in os.walk(dataset_dir):
    for file in sorted(files):
        if file.endswith('.csv') and not file.endswith('_scaled.csv'):  # Process only unscaled .csv files
            csv_path = os.path.join(root, file)
            if 'train' in file:
                # Handle training data
                scaler_path = os.path.join(root, 'scaler_' + file.replace('.csv', '.joblib'))
                scaled_train_df = load_and_scale_data(csv_path, save_scaler_path=scaler_path)
                # Save the scaled training data
                scaled_csv_path = csv_path.replace('.csv', '_scaled.csv')
                scaled_train_df.to_csv(scaled_csv_path, index=False)
            elif 'test' in file:
                # Handle testing data
                scaler_path = os.path.join(root, 'scaler_' + file.replace('_test.csv', '_train.joblib'))
                scaler = joblib.load(scaler_path) if os.path.exists(scaler_path) else None
                scaled_test_df = load_and_scale_data(csv_path, scaler=scaler)
                # Save the scaled testing data
                scaled_csv_path = csv_path.replace('.csv', '_scaled.csv')
                scaled_test_df.to_csv(scaled_csv_path, index=False)

            # Delete the original unscaled .csv file
            os.remove(csv_path)


In [8]:
import os
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule(epoch, lr):
    """Learning rate schedule function.
    
    Decreases the learning rate by a factor of 0.1 every 10 epochs.
    """
    if epoch % 10 == 0 and epoch > 0:
        return lr * 0.1
    return lr

# Directory containing your scaled CSV files
csv_directory = '/home/ubuntu/dds_paper/DDS_Paper/data/DDS_Data_SEU/data/csvs'

def load_data(train_path, test_path):
    # Load datasets
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # Split into features and labels
    X_train, y_train = train_df.iloc[:, :-1], train_df.iloc[:, -1]
    X_test, y_test = test_df.iloc[:, :-1], test_df.iloc[:, -1]

    # Encode labels
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)

    y_train_onehot = to_categorical(y_train_encoded)
    y_test_onehot = to_categorical(y_test_encoded)

    # Reshape for LSTM [samples, timesteps, features]
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

    return X_train, y_train_onehot, X_test, y_test_onehot

def create_model(input_shape, num_classes):
    model = Sequential([
        LSTM(100, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(100, return_sequences=True),
        Dropout(0.2),
        LSTM(100),
        Dropout(0.2),
        Dense(100, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile model with the initial learning rate
    model.compile(optimizer=Adam(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Iterate through the directory to find matching scaled train and test pairs
for file in os.listdir(csv_directory):
    if file.endswith("_train_scaled.csv"):
        base_name = file.replace("_train_scaled.csv", "")
        test_file = base_name + "_test_scaled.csv"

        if test_file in os.listdir(csv_directory):
            train_path = os.path.join(csv_directory, file)
            test_path = os.path.join(csv_directory, test_file)

            print(f"Processing: {base_name}")
            X_train, y_train, X_test, y_test = load_data(train_path, test_path)

            # Create model
            model = create_model(input_shape=(X_train.shape[1], X_train.shape[2]), num_classes=y_train.shape[1])
            # Create an instance of the LearningRateScheduler callback
            lr_scheduler = LearningRateScheduler(lr_schedule, verbose=0)
            # Early stopping to avoid overfitting
            early_stopping = EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)

            # Train the model
            model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, batch_size=10000, callbacks=[early_stopping, lr_scheduler], verbose=1)

            # Here, you might want to save the model and any evaluation metrics you care about
            # model.save(f'model_{base_name}.h5')


Processing: PGB_30_4


/home/ubuntu/miniconda3/envs/ltn/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.1825 - loss: 2.2722 - val_accuracy: 0.2158 - val_loss: 4.7243 - learning_rate: 0.1000
Epoch 2/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.3324 - loss: 1.6752 - val_accuracy: 0.2252 - val_loss: 4.9041 - learning_rate: 0.1000
Epoch 3/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy: 0.3751 - loss: 1.5864 - val_accuracy: 0.2259 - val_loss: 5.9388 - learning_rate: 0.1000
Epoch 4/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 0.3882 - loss: 1.5582 - val_accuracy: 0.2243 - val_loss: 5.9462 - learning_rate: 0.1000
Epoch 5/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 0.3956 - loss: 1.5448 - val_accuracy: 0.2225 - val_loss: 7.1452 - learning_rate: 0.1000
Epoch 6/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - accuracy: 0.4011 - loss: 1.5347 - val_accuracy: 0.2242 - val_loss: 7.5675 - learning_rate: 0.1000
Epoch 7/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - accuracy: 0.4051 - loss: 

/home/ubuntu/miniconda3/envs/ltn/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 12s 97ms/step - accuracy: 0.2209 - loss: 1.9566 - val_accuracy: 0.2333 - val_loss: 7.6980 - learning_rate: 0.1000
Epoch 2/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - accuracy: 0.3632 - loss: 1.5742 - val_accuracy: 0.2127 - val_loss: 10.3114 - learning_rate: 0.1000
Epoch 3/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 0.3773 - loss: 1.5405 - val_accuracy: 0.2183 - val_loss: 9.3751 - learning_rate: 0.1000
Epoch 4/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 0.3848 - loss: 1.5273 - val_accuracy: 0.2162 - val_loss: 8.8876 - learning_rate: 0.1000
Epoch 5/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy: 0.3914 - loss: 1.5189 - val_accuracy: 0.2201 - val_loss: 11.2394 - learning_rate: 0.1000
Epoch 6/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.3970 - loss: 1.5125 - val_accuracy: 0.2196 - val_loss: 10.4883 - learning_rate: 0.1000
Epoch 7/1000
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.4013 - loss

KeyboardInterrupt: 